In [1]:
#from xml.dom import minidom
from bs4 import BeautifulSoup
import re
import os

In [ ]:
#Parse sheet data for sources
sheet_data_file = 'COVID_tracking_sheet1.tsv'   #Kato link, source
sheet_data = [line.strip().split('\t') for line in open(sheet_data_file,'r').readlines()]

job_sources = {}  #project-id, source
for job_info in sheet_data:
    job_kato_link = job_info[0]
    project_id = job_kato_link.split("/")[4]
    print(project_id + ', ', end='')
    if len(job_info) == 2:
        job_source = job_info[1]
        job_sources[project_id] = job_source
    else:
        job_sources[project_id] = ""  

In [ ]:
#Parse MatecatLanguagePairs.csv output to 
language_pairs_file = 'MatecatLanguagePairs.csv'   #Kato link, source
job_data = [line.strip().split(',') for line in open(language_pairs_file,'r').readlines()[1:]]

tmx_links = {}  #project-id, tmxlink
job_2_source = {} #matecat-id, source
job_2_projectid = {}
project_ids = set()
for _,matecat_id_job, matecat_id_job_password, matecat_langpair, project_id, _, _ in job_data:
    tmx_links[project_id] = 'https://tm.translatorswb.org/TMX/' + matecat_id_job + '/' + matecat_id_job_password
    job_2_source[matecat_id_job] = job_sources[project_id]
    job_2_projectid[matecat_id_job] = project_id
    project_ids.add(int(project_id))

In [2]:
def init_file(file):
    with open(file,'w') as f:
        f.write('')
            
def append_list_to_file(l, file):
    with open(file,'a') as f:
        for e in l:
            f.write(e + '\n')
    
def clean_sent(text):
    text = text.replace("\n", '')
    text = text.replace("&#10;", '')
    text = text.replace("\r", '')
    text = text.replace("\t", ' ')
    
    tag_cleaner = re.compile('<.*?>')
    clean_text = re.sub(tag_cleaner, '', text)
    return clean_text

In [3]:
job2doc_data_file = 'Job_docfilename.csv'   #Kato link, source
job2doc_data = {line.strip().split('\t')[0]:line.strip().split('\t')[1] for line in open(job2doc_data_file,'r').readlines()}

job2doc_id = {}
curr_doc_id = 0
curr_doc = ''
for job_id in job2doc_data:
    doc_name = job2doc_data[job_id]
    if not doc_name == curr_doc:
        curr_doc_id += 1
        curr_doc = doc_name
    #doc_ids.append(curr_doc_id)
    job2doc_id[job_id] = curr_doc_id
    
    
doc_id2doc_name = {}
for job_id in job2doc_id:
    doc_id = job2doc_id[job_id]
    doc_id2doc_name[doc_id] = job2doc_data[job_id]

In [6]:
from shutil import copyfile

doc_id2indexeddoc_name = {}
for doc_id in doc_id2doc_name:
    new_name = str(doc_id) + "_" + doc_id2doc_name[doc_id]
    #copyfile("doc/" + doc_id2doc_name[doc_id], "doc_indexed/" + new_name)
    doc_id2indexeddoc_name[doc_id] = new_name

In [25]:
#Output to one file with format
#sourceLang | targetLang | sourceString | targetString | stringID | url | License

tmx_dir = 'tmx'
tmx_list_file = "tmx_list.txt"
tmx_info = [line.strip().split('\t') for line in open(tmx_list_file,'r').readlines()]

output_file = 'TWB-COVID-TM.tsv'

total_lines = 0
lang_lines = {}
lang_docs = {}
doclangid_sentcount = {}
jobid_sentcount = {}
jobid_to_doclangid = {}

id_prefix = "TWB"
license = "TWB custom license"

with open(output_file, 'w') as f_out:
    for tmx_file, langtagpair in tmx_info:
        job_id = tmx_file.split('.')[0]
        tmx_path = os.path.join(tmx_dir, tmx_file)
        langtag1 = langtagpair.split("|")[0]
        langtag2 = langtagpair.split("|")[1]

        lang1 = langtag1.split("-")[0]
        lang2 = langtag2.split("-")[0]
        
        doclangid = id_prefizx + "_" + str(job2doc_id[job_id]) + "_" + langtag2
        jobid_to_doclangid[job_id] = doclangid

        sents_lang1 = []
        sents_lang2 = []

        with open(tmx_path) as f:
            content = f.read()

        soup = BeautifulSoup(content)

        skip_file = False
        segments = soup.findAll("tu")
        for segment in segments:
            try:
                lang1_text = segment.find("tuv", {"xml:lang": langtag1}).seg.text
                lang2_text = segment.find("tuv", {"xml:lang": langtag2}).seg.text

                sents_lang1.append(clean_sent(lang1_text))
                sents_lang2.append(clean_sent(lang2_text))
            except Exception as e:
                print(e)
                skip_file = True
                break

        if skip_file:
            print("Problem in file ", tmx_path)
            continue

        if not len(sents_lang1) == len(sents_lang2):
            print("# sentences don't match in file ", tmx_path)
            
        doclangid_sentcount[doclangid] = len(sents_lang1)
        jobid_sentcount[job_id] = len(sents_lang1)

        for sent_lang1, sent_lang2 in zip(sents_lang1, sents_lang2):
            total_lines += 1
            string_id = doclangid + ":" + str(total_lines)
#             if job_id in matecat_2_source:
#                 source = matecat_2_source[job_id]
#             else:
#                 #print("No source for", job_id)
#                 source = ""
            line = '\t'.join([lang1, lang2, sent_lang1, sent_lang2, string_id, doc_id2indexeddoc_name[job2doc_id[job_id]], license])
            f_out.write(line + '\n')
            #print(line)

print("Total sents: ", total_lines)

Total sents:  9229


In [15]:
jobid_sentcount['10867']

186

In [27]:
#For the sheet
#Job ID,Doc ID,DocLang ID,Doc name,Translation direction,Word Count,Date Entered
enter_date = '5/1/2020'
with open('doc_translation_languages.csv', 'w') as f:
    for tmx_file, langpair in tmx_info:
        job_id = tmx_file.split('.')[0]
        f.write(job_id + "," + str(job2doc_id[job_id]) + ',' + str(jobid_to_doclangid[job_id]) + ',' + doc_id2indexeddoc_name[job2doc_id[job_id]] 
                + "," + langpair + ',' + str(jobid_sentcount[job_id]) + ',' + enter_date + '\n') 

In [55]:
#proofcheck
covid_data = [line.strip().split('\t') for line in open(output_file,'r').readlines()]

bostu = False
for d in covid_data:
#     print(d[4])
    if bostu:
        print(d[4])
    if not d[4] or d[4].isspace():
        bostu=True
    

In [ ]:
#Output to seperate files
tmx_dir = 'tmx'
job_list_file = "Job_ID_PASS.csv"
jobs = [line.strip().split('\t') for line in open(job_list_file,'r').readlines()]

parallel_out_dir = 'parallel'

total_lines = 0
lang_lines = {}
lang_docs = {}

for job_id, _, langtagpair in jobs:
    tmx_file = job_id + '.tmx'
    tmx_path = os.path.join(tmx_dir, tmx_file)
    langtag1 = langtagpair.split("|")[0]
    langtag2 = langtagpair.split("|")[1]
    
    lang1 = langtag1.split("-")[0]
    lang2 = langtag2.split("-")[0]
    

    #print(tmx_path + " "+ lang1 + " " + lang2)
    
    sents_lang1 = []
    sents_lang2 = []
    
    with open(tmx_path) as f:
        content = f.read()
        
    soup = BeautifulSoup(content)
    
    skip_file = False
    segments = soup.findAll("tu")
    for segment in segments:
        try:
            lang1_text = segment.find("tuv", {"xml:lang": langtag1}).seg.text
            lang2_text = segment.find("tuv", {"xml:lang": langtag2}).seg.text

            sents_lang1.append(clean_sent(lang1_text))
            sents_lang2.append(clean_sent(lang2_text))
        except:
            skip_file = True
            break
    
    if skip_file:
        print("Problem in file ", tmx_path)
        continue
        
    if not len(sents_lang1) == len(sents_lang2):
        print("# sentences don't match in file ", tmx_path)

    langs = [lang1, lang2]
    langs.sort()
    langpair = '-'.join(langs)
    out_dir = os.path.join(parallel_out_dir, lang2)
    
    lang1_file = os.path.join(out_dir, langpair + '.' + lang1)
    lang2_file = os.path.join(out_dir, langpair + '.' + lang2)
    
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
        init_file(lang1_file)
        init_file(lang2_file)
        lang_lines[lang2] = 0
        lang_docs[lang2] = 0

    append_list_to_file(sents_lang1, lang1_file)
    append_list_to_file(sents_lang2, lang2_file)

    total_lines += len(sents_lang1)
    lang_lines[lang2] += len(sents_lang1)
    lang_docs[lang2] += 1
    
print("Total sents: ", total_lines)
print("Sents for each lang")
print(lang_lines)
print("Docs for each lang")
print(lang_docs)

In [ ]:
sorted_lang_lines = {k: v for k, v in sorted(lang_lines.items(), key=lambda item: item[1], reverse=True)}
for l in sorted_lang_lines:
    print(l + ',' + str(lang_lines[l]))

In [ ]:
for i in range(0,1000,50):
    print(i, end=',')